### 第二引数で書き込みモードを指定

In [2]:
with open("some.txt", "w") as f:
    f.write("python")

### 例外処理もクローズされた

In [3]:
f.closed

True

# 【コンテキストマネージャーの実装】

## ■`__enter__()`、`__exit__()` withの前後に呼ばれるメソッド

### このクラスのインスタンスがコンテキストマネージャー

In [4]:
class ContextManager:
    # 前処理を実装
    def __enter__(self):
        print("__enter__ was called")

    # 後処理を実装
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"{exc_type=}")
        print(f"{exc_value=}")
        print(f"{traceback=}")

### withブロックが正常の場合は
### `__exit__()`の引数はすべてNone

In [5]:
with ContextManager():
    print("inside the block")

__enter__ was called
inside the block
exc_type=None
exc_value=None
traceback=None


## ■with分と例外処理

### withブロックナイで例外が発生した場合は
### その情報が__exit__()に渡される

In [7]:
with ContextManager():
    1 / 0

__enter__ was called
exc_type=<class 'ZeroDivisionError'>
exc_value=ZeroDivisionError('division by zero')
traceback=<traceback object at 0x00000147DD5F9940>


ZeroDivisionError: division by zero

### ■asキーワード - `__enter__()`の戻り値を利用する

In [11]:
class ContextManager:
    # 戻り値がasキーワードに渡される
    def __enter__(self):
        return 1

    def __exit__(self, exc_type, exc_value, traceback):
        pass

In [12]:
with ContextManager() as f:
    print(f)

1


In [16]:
class Point:
    def __init__(self, **kwargs):
        self.value = kwargs

    def __enter__(self):
        print("__enter__ was called")
        return self.value  # as説で渡される

    def __exit__(self, exc_type, exc_value, traceback):
        print("__exit__ was called")
        print(self.value)

In [17]:
with Point(x=1, y=2) as p:
    print(p)
    p["z"] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}


## ■contextlib.contextmanagerでシンプルに実装する

In [20]:
from contextlib import contextmanager


@contextmanager
def point(**kwargs):
    print("__enter__ was called")
    value = kwargs
    try:
        # yield式より上が前処理
        # valueがasキーワードに渡される
        yield value
    # yield式より下が後処理
    except Exception as e:
        # エラー時はこちらも呼ばれる
        print(e)
        raise
    finally:
        print("__exit__ was called")
        print(value)

In [21]:
with point(x=1, y=2) as p:
    print(p)
    p["z"] = 3

__enter__ was called
{'x': 1, 'y': 2}
__exit__ was called
{'x': 1, 'y': 2, 'z': 3}
